<a href="https://colab.research.google.com/github/Sabari-Vijayan/Project_Useless_KTULLM/blob/main/LoadModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install unsloth transformers

In [2]:
!pip install bitsandbytes
!pip install torch # Install PyTorch
from transformers import AutoModel, BitsAndBytesConfig
import torch  # Import the torch module

In [5]:
!pip install accelerate
!pip install peft
!pip install transformers

from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

In [6]:
model_id = "ClumsyAahDeveloper/new_model"  # Replace with your model ID

# Load the base model and tokenizer
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True,  torch_dtype=torch.float16, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Load the LoRA weights using PeftModel
model = PeftModel.from_pretrained(model, model_id, torch_dtype=torch.float16, device_map='auto')

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.2k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/464 [00:00<?, ?B/s]

In [11]:
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
       model_name="ClumsyAahDeveloper/new_model",  # Your Hugging Face model ID
       max_seq_length=2048,
       dtype=torch.float16,
       load_in_4bit=True,
   )
FastLanguageModel.for_inference(model)  # Prepare for inference

==((====))==  Unsloth 2024.11.7: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

Unsloth 2024.11.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128255)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lor

In [22]:
messages = """Below are some instructions that describe some tasks. Write responses that appropriately completes the request.

### Instruction:
{INPUT}

### Response:
{OUTPUT}"""

In [28]:
from unsloth import FastLanguageModel
from transformers import TextStreamer

# ... (previous code for loading the model, setting dtype/load_in_4bit) ...

tokenizer.add_special_tokens({'additional_special_tokens': ['<|endoftext|>']})
tokenizer.eos_token = None # Assuming you don't want eos behavior

# Define a more structured chat template
chat_template = """<s>[INST] <<SYS>>
You are a helpful and harmless AI assistant.
<</SYS>>

{history}

[/INST]"""

def format_history(messages):
    history = ""
    for message in messages:
        if message["role"] == "user":
            history += f"User: {message['content']}\n"
        elif message["role"] == "assistant":
            history += f"Assistant: {message['content']}\n"
    return history

# Example usage:
messages = []
while True:
    user_input = input("You: ")
    messages.append({"role": "user", "content": user_input})

    formatted_history = format_history(messages)
    input_ids = tokenizer(
        chat_template.format(history=formatted_history),
        return_tensors="pt",
    ).input_ids.to("cuda")

    text_streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
    _ = model.generate(
        input_ids,
        streamer=text_streamer,
        max_new_tokens=128,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.7,
    )
    print()  # Print a newline for better formatting

    # Extract and print the assistant's response
    assistant_response = text_streamer.text.strip()  # Get the generated text
    messages.append({"role": "assistant", "content": assistant_response})
    print(f"Assistant: {assistant_response}")

You: hi


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


AttributeError: 'TextStreamer' object has no attribute 'text'